# Notebook to check direct db calls against Bovas's NAV report code

In [1]:
%reload_ext autoreload
%autoreload 
import sys, os
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root_path not in sys.path:
    sys.path.append(project_root_path)
print(project_root_path)

import datetime as dt
import numpy as np
import pandas as pd
import plotly.express as px

import pymd 
import nav.utils
from qpt_historic_pos.impl.utils.times import ChicagoTimeZone, UtcTimeZone
import qpt_stress_test.apps.generate_daily_summary as generate_daily_summary
import qpt_stress_test.services.positions as positions
import qpt_stress_test.core.config as config
import qpt_stress_test.core.qpt_config as qpt_config
import qpt_stress_test.db.tasks as db_tasks
import qpt_stress_test.db.repositories.drivers.pyodbc as pyodbc
import qpt_stress_test.db.repositories.drivers.sqlalchemy as sqlalchemy
import qpt_stress_test.db.repositories.qpt_pg as qpt_pg
import qpt_stress_test.db.repositories.qpt_mssql as qpt_mssql
import qpt_stress_test.db.repositories.databricks as db_trading
from qpt_stress_test.db.tasks import bfc_rds_sqlalchemy_engine_factory, sv_awoh_dw01_pyodbc_connection_factory,  gdt_cluster_databricks_connection_factory

from importlib import reload
reload(positions)
reload(config)
reload(qpt_config)
reload(db_tasks)

pymd.enable_logging()

c:\Users\skingham\Projects\galaxysk\qpt_stress_test


## Setup config and repo objects

In [2]:
# Set dates, etc, for report generation
nav_date = dt.date(2023, 1, 31)
nav_00utc = dt.datetime.combine(nav_date + dt.timedelta(days=1), dt.time(hour=0, minute=0, second=0))
nav_ctz = UtcTimeZone.localize(dt.datetime.combine(nav_date + dt.timedelta(days=1), dt.time(hour=0, minute=0, second=0))).astimezone(ChicagoTimeZone)
derivs_utc_datetime = ChicagoTimeZone.localize(dt.datetime.combine(nav_date, dt.time(hour=16, minute=0, second=0))).astimezone(UtcTimeZone)

In [3]:
%%capture
mktdata_repo = qpt_mssql.MarketDataRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)
trading_repo = qpt_mssql.TradingRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)
operations_repo = qpt_mssql.OperationsRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)
exchange_repo = qpt_mssql.RawDataRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)

## Get some info about the clients traded today

In [5]:
%%capture
valid_client_list = [entry['Client'] for entry in trading_repo.get_daily_client_names(nav_date).as_map().values()]
valid_account_list = [entry['Account'] for entry in trading_repo.get_daily_accounts(nav_date).as_map().values()]
ops_account_list = [entry['Account'] for entry in operations_repo.get_eod_account_names(nav_date).as_map().values()]
trading_eod_balances = trading_repo.get_eod_balances(nav_date).as_dataframe()

# Filter out invalid accounts and clients
filtered_trading_balances = trading_eod_balances.loc[(trading_eod_balances['Account'].isin(valid_account_list)) & (trading_eod_balances['Client'].isin(valid_client_list))].reset_index(drop=True)


AttributeError: 'TradingRepository' object has no attribute 'get_eod_balances'

## Just look at LINK strategies

In [ ]:
# Link client info
link_client_df = filtered_trading_balances.loc[filtered_trading_balances['Symbol'].apply(lambda s: 'LINK' in s.upper())]

# Make the each currency/product position into a column
link_client2_df = link_client_df.pivot(index=['TradeDate', 'Client', 'Endpoint', 'Account', 'Symbol'], columns='Product')['EodPosition'].reset_index().rename_axis(None, axis='columns').fillna(0)
display(link_client2_df)

# CC_TRlinkFSpread1 does not net out to delta neutral
display(link_client2_df[['TradeDate','Client', 'Symbol', 'BLINK', 'LINK', 'USD', 'USDT']].groupby(['TradeDate','Client', 'Symbol']).sum())

,TradeDate,Client,Endpoint,Account,Symbol,BLINK,LINK,USD,USDT
0,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,LINKUSDT,0.00,-11.81,0.00,79.53
1,2023-01-31,CC_FEE,HUBI,HUBI-E,LINKUSDT,0.00,"-6,960.53",0.00,"134,758.84"
2,2023-01-31,CC_FEE,HUBI,HUBI-M,LINKUSDT,0.00,-284.66,0.00,"7,428.07"
3,2023-01-31,CC_FEE_BULL,BINE,BINE-2-S1-E,LINKUSDT,0.00,353.31,0.00,"-2,372.60"
4,2023-01-31,CC_FEE_BULL,HUBI,HUBI-E,LINKUSDT,0.00,"3,533.59",0.00,"-25,182.59"
5,2023-01-31,CC_FUNDING,HUBI,HUBI-E,LINKUSDT,0.00,160.01,0.00,"-8,313.48"
6,2023-01-31,CC_Interest2020,BINE,BINE-2-S1-E,LINKUSDT,0.00,-2.70,0.00,40.58
7,2023-01-31,CC_Interest2020,HUBI,HUBI-E,LINKUSDT,0.00,"6,155.61",0.00,"-122,314.06"
8,2023-01-31,CC_Interest2020,LEND,LEND-BTGO,LINKUSD,0.00,"-2,889.03",0.00,0.00
9,2023-01-31,CC_Interest2020,LEND,LEND-GACM,LINKUSD,0.00,"-1,791.11",0.00,0.00


BLINK      LINK        USD  \
TradeDate  Client            Symbol                                      
2023-01-31 CC_FEE            LINKUSDT        0.00 -7,257.00       0.00   
           CC_FEE_BULL       LINKUSDT        0.00  3,886.90       0.00   
           CC_FUNDING        LINKUSDT        0.00    160.01       0.00   
           CC_Interest2020   LINKUSD         0.00 -6,125.14       0.00   
                             LINKUSDT        0.00  6,152.91       0.00   
           CC_MARGIN_WOOX    LINKUSDT        0.00      0.00       0.00   
           CC_TRlinkFSpread1 BLINKUSDT -37,004.40      0.00       0.00   
                             LINKUSD         0.00 25,740.85  67,683.76   
                             LINKUSDT        0.00 -1,111.96       0.00   
           CC_TRlinkGSpread3 LINKUSD         0.00  1,071.35 -12,459.56   
                             LINKUSDT        0.00 -1,071.74       0.00   

                                              USDT  
TradeDate  Client            Symbol                 
2023-01-31 CC_FEE            LINKUSDT   142,266.44  
           CC_FEE_BULL       LINKUSDT   -27,555.18  
           CC_FUNDING        LINKUSDT    -8,313.48  
           CC_Interest2020   LINKUSD          0.00  
                             LINKUSDT  -122,273.48  
           CC_MARGIN_WOOX    LINKUSDT        -0.34  
           CC_TRlinkFSpread1 BLINKUSDT  415,248.38  
                             LINKUSD          0.00  
                             LINKUSDT    -5,428.24  
           CC_TRlinkGSpread3 LINKUSD          0.00  
                             LINKUSDT    12,622.31

## How big is the filtering problem? 

In [ ]:
# Find any client or account that is not used in yesterdays transaction fills or operations account balance reports
invalid_client_list = sorted(set(trading_eod_balances.loc[~trading_eod_balances['Client'].isin(valid_client_list), 'Client'].to_list()))
invalid_account_list = sorted(set(trading_eod_balances.loc[~trading_eod_balances['Account'].isin(valid_account_list), 'Account'].to_list()))

print("Valid client names:" + str(sorted(valid_client_list))) 
print("\nInvalid client names:" +str(sorted(invalid_client_list))) 
print("\nValid account names:" +str(sorted(valid_account_list))) 
print("\nInvalid account names:" +str(sorted(invalid_account_list))) 

Valid client names:['CC_BINEmmRebate', 'CC_FEE', 'CC_FEE_BULL', 'CC_FUNDING', 'CC_FUNDING_DYDX', 'CC_Interest2020', 'CC_MARGIN_BINE', 'CC_MARGIN_OKEX', 'CC_MARGIN_WOOX', 'CC_TRaaveGSpread3', 'CC_TRaaveGSpread4', 'CC_TRadaGSpread3', 'CC_TRadaGSpread4', 'CC_TRalgoGSpread3', 'CC_TRavaxGSpread3', 'CC_TRaxsGSSpread1', 'CC_TRaxsGSSpread2', 'CC_TRbatGSpread3', 'CC_TRbchFRateSpread1', 'CC_TRbchGSpread3', 'CC_TRbchRateSpread5', 'CC_TRbtcB2RateSpread30', 'CC_TRbtcBARateSpread21', 'CC_TRbtcBFRateSpread21', 'CC_TRbtcBFTRateSpread21', 'CC_TRbtcBGSpread1', 'CC_TRbtcBRateSpread19', 'CC_TRbtcBRateSpread191', 'CC_TRbtcBRateSpread61', 'CC_TRbtcBT2RateSpread30', 'CC_TRbtcBTRateSpread192', 'CC_TRbtcBTRateSpread196', 'CC_TRbtcBTRateSpread21', 'CC_TRbtcDARateSpread21', 'CC_TRbtcDBRateSpread20', 'CC_TRbtcDFRateSpread21', 'CC_TRbtcDFSpread1', 'CC_TRbtcDFSpread2', 'CC_TRbtcDRateSpread19', 'CC_TRbtcDRateSpread197', 'CC_TRbtcDRateSpread20', 'CC_TRbtcDRateSpread60', 'CC_TRbtcERateSpread3', 'CC_TRbtcETSpread1', 'C

In [ ]:
display(filtered_trading_balances)

,TradeDate,Client,Endpoint,Account,Symbol,Product,EodPosition
0,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,BCHUSDT,BCH,-0.72
1,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,BCHUSDT,USDT,96.60
2,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,EOSUSDT,EOS,-525.40
3,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,EOSUSDT,USDT,541.08
4,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,ETCUSDT,ETC,-3.87
5,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,ETCUSDT,USDT,85.30
6,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,LINKUSDT,LINK,-11.81
7,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,LINKUSDT,USDT,79.53
8,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,LTCUSDT,LTC,-6.33
9,2023-01-31,CC_FEE,BINE,BINE-2-S1-E,LTCUSDT,USDT,532.30
